<h1 align='center'> CodeForces Visualizer </h1>

In [1]:
import os
import json
import requests
import pandas as pd
import numpy as np
import datetime
from copy import deepcopy

* Base url for api call for each user.

In [2]:
# give information about user's submission history
URL1 = "https://codeforces.com/api/user.status?handle="
# give informaiton about user's competition history
URL2 = "https://codeforces.com/api/user.rating?handle="

* Helper function to retrive data from server

In [3]:
def getSubData(uname):
    return json.loads(requests.get(URL1 + uname).content)['result']

def getCompiData(uname):
    return json.loads(requests.get(URL2 + uname).content)['result']

In [4]:
user = 'tourist'
subData = getSubData(user)
compiData = getCompiData(user)

In [5]:
print("Total Number of sumbission for user %s is %d" % (user,len(subData)))

Total Number of sumbission for user tourist is 1949


In [6]:
print("Total Number of compitition participated for user %s is %d" % (user,len(compiData)))

Total Number of compitition participated for user tourist is 148


# Data Mapping and Transformation

In [7]:
# map compiData based on their contesId
def mapCompiData(cdata):
    data = {}
    cdata_copy = deepcopy(cdata)
    for row in cdata_copy:
        id_ = row.pop('contestId')
        data[id_] = row
    return data

In [8]:
def mapSubmissions(sdata):
    data = {}
    sdata_copy = deepcopy(sdata)
    for sub in sdata_copy:
        name = str(sub['problem']['contestId']) + "_" + sub['problem']['index']
        if name not in data:
            data[name] = []
        data[name].append(sub)
    return data

In [9]:
# adding each submission to corresponding compitition
def addSubToContest(cdata, sdata):
    contest_submissions = [d for d in sdata if d['author']['participantType'] in ['CONTESTANT', 'OUT_OF_COMPETITION'] ]
    cdata_copy = deepcopy(cdata)
    for sub in contest_submissions:
        if sub['contestId'] not in cdata_copy:
            continue
        contest = cdata_copy[sub['contestId']]
        
        if 'submissions' not in contest:
            contest['submissions'] = []
        contest['submissions'].append(sub)
    return cdata_copy

In [10]:
full_contest_data = addSubToContest(mapCompiData(compiData), subData)

# Plot 1:
* This plot shows comparison between two users compititon rating.
* It will allow user to track and compare their progress with other user from time to time.
* Each user starts with 1500 rating.
* This will allow one team member compare their rating and improve accordinly.

### Why we need this plot?
* Contest history gives overall overview on how much and how fast you progressed over time.
* Here we not only giving overview of competition rating change but other statistics related to each contest like number of submission in that contest, average number of try to took solve a problem etc.

In [11]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

In [12]:
'''
* get_stats and get_stats_default are helper method to get data for hover text.
* both _default method is dependent on get_stats method, update in get_stats will
  also leads to update in _default method.
'''

def get_stats(date, data):
    for d in data.values():
        if date == datetime.datetime.fromtimestamp(float(d['ratingUpdateTimeSeconds'])):
            return [d['newRating'], len(d['submissions']), d['contestName']]
        
# this is default set of display stats.
def get_stats_default(date_, data):
    if len(data)>0:
        return (date_, data[-1][1], -1, 'Not Participated')
    else:
        return (date_, 1500, -1, 'Not Participated')
        
def plot_two_rating(username1,username2):
    compiData1 = getCompiData(username1)
    compiData2 = getCompiData(username2)
    
    subData1 = getSubData(username1)
    subData2 = getSubData(username2)
    
    full_contest_data1 = addSubToContest(mapCompiData(compiData1), subData1)
    full_contest_data2 = addSubToContest(mapCompiData(compiData2), subData2)
    
    date_user1 = sorted(list(set([datetime.datetime.fromtimestamp(float(i['ratingUpdateTimeSeconds'])) for i in full_contest_data1.values()])))
    date_user2 = sorted(list(set([datetime.datetime.fromtimestamp(float(i['ratingUpdateTimeSeconds'])) for i in full_contest_data2.values()])))

    total_dates = sorted(list(set(date_user1 + date_user2)))
    
    data1 = []
    data2 = []
    for date_ in total_dates:
        if date_ in date_user1:
            data1.append((date_, *get_stats(date_,full_contest_data1)))
        else:
            data1.append(get_stats_default(date_, data1))

        if date_ in date_user2:
            data2.append((date_, *get_stats(date_,full_contest_data2)))
        else:
            data2.append(get_stats_default(date_, data2))

    pldata1 = go.Scatter(
        x=[x[0] for x in data1], 
        y=[x[1] for x in data1], 
        text = ["Total Submission In contest: " + str(v[2]) 
                + "<br>Rating: " 
                + str(v[1])
                + "<br>Name: "
                + v[3]
                for v in data1],
        hoverinfo ='text',
        name=username1
    )
    pldata2 = go.Scatter(
        x=[x[0] for x in data2], 
        y=[x[1] for x in data2], 
        text = ["Total Submission In contest: " + str(v[2]) 
                + "<br>Rating: " 
                + str(v[1])
                + "<br>Name: "
                + v[3]
                for v in data2],
        hoverinfo ='text',
        name=username2
    )
    
    # setting up the 
    layout = dict(
        title='Contest rating comparision between ' + username1 + " and " + username2,
        xaxis=dict(
            rangeselector=dict(
                buttons=list([
                    dict(count=1,
                         label='1m',
                         step='month',
                         stepmode='backward'),
                    dict(count=6,
                         label='6m',
                         step='month',
                         stepmode='backward'),
                    dict(count=1,
                        label='1y',
                        step='year',
                        stepmode='backward'),
                    dict(step='all')
                ])
            ),
            rangeslider=dict(
                visible = True
            ),
            type='date'
        )
    )
    fig = dict(data=[pldata1,pldata2], layout=layout)
    return iplot(fig, filename = "Manually Set Range")

In [107]:
plot_two_rating('MoHib85','tourist')

# Plot 2
* This plot will explore users average number of submisison required over different topic.

### Why we need this plot?
* This plot gives great analyz about users strength and weakness.
* From average number of tries required for a given topic graph classify into 3 category
    - already mastered topics
    - topics on which you have good understanding
    - topics which you need to practive more

In [123]:
def get_topic_stat(data):
    d = {}
    for subs in data.values():
        tags = subs[0]['problem']['tags']
        subs.sort(key = lambda x: x['relativeTimeSeconds'])
        
        for t in tags:
            
            if t not in d:
                d[t] = [1,1]
            flag = 0    
            for sub in subs:
                if flag:
                    break
                if sub['verdict'] == 'OK':
                    flag = 1
                    d[t][1] += 1
                else:
                    if sub['verdict'] != 'SKIPPED':
                        d[t][0] += 1
    return d

In [141]:
def topic_stats_scatter_plot(username):
    subData = getSubData(username)
    mapped_sub_data = mapSubmissions(subData)
    topic_stats = get_topic_stat(mapped_sub_data)
    avg_topic_stats = dict(sorted(list(zip(topic_stats.keys(), list(map(lambda x: [((x[1]+x[0])/x[1]), x[1]+x[0]], topic_stats.values())))), key=lambda x:x[1][0]))
    
    max_try = max(avg_topic_stats.values())[0]
    min_try = 1.0
    q = (max_try - min_try)/3.0
    
    q1 = [(k,v) for k,v in avg_topic_stats.items() if v[0] <= 1+q]
    q2 = [(k,v) for k,v in avg_topic_stats.items() if v[0] <= 1+q+q and v[0] > 1+q]
    q3 = [(k,v) for k,v in avg_topic_stats.items() if v[0] > 1+q+q]    
    
    trace1 = go.Scatter(
        x = [v[1][0] for v in q1],
        y = [v[1][1] for v in q1],
        text = ["Topic: "
                + v[0]
                + "<br>Total Submission of the Topic: " 
                + str(v[1][1])  
                + "<br>Average Try to get AC: "
                + str(v[1][0])
                for v in q1],
        hoverinfo ='text',
        name = 'Mastered',
        mode = 'markers',
        marker = dict(
            size = 10,
            color = 'rgba(152, 0, 0, .8)',
            line = dict(
                width = 3,
                color = 'rgb(0, 0, 0)'
            )
        )
    )
    
    trace2 = go.Scatter(
        x = [v[1][0] for v in q2],
        y = [v[1][1] for v in q2],
        text = ["Topic: "
                + v[0]
                + "<br>Total Submission of the Topic: " 
                + str(v[1][1])  
                + "<br>Average Try to get AC: "
                + str(v[1][0])
                for v in q2],
        hoverinfo ='text',
        name = 'Ok',
        mode = 'markers',
        marker = dict(
            size = 10,
            color = 'rgba(152, 0, 255, .8)',
            line = dict(
                width = 3,
                color = 'rgb(0, 0, 255)'
            )
        )
    )
    
    trace3 = go.Scatter(
        x = [v[1][0] for v in q3],
        y = [v[1][1] for v in q3],
        text = ["Topic: "
                + v[0]
                + "<br>Total Submission of the Topic: " 
                + str(v[1][1])  
                + "<br>Average Try to get AC: "
                + str(v[1][0])
                for v in q3],
        hoverinfo ='text',
        name = 'Focus On This',
        mode = 'markers',
        marker = dict(
            size = 10,
            color = 'rgba(152, 255, 0, .8)',
            line = dict(
                width = 3,
                color = 'rgb(0, 255, 0)'
            )
        )
    )
    data = [trace1, trace2, trace3]

    layout = dict(title = 'Styled Scatter',
                  yaxis = dict(zeroline = False),
                  xaxis = dict(zeroline = False)
                 )

    fig = dict(data=data)
    return iplot(fig, filename='styled-scatter')

In [142]:
topic_stats_scatter_plot('guille')